In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

## Data completeness

In [2]:
df_clicks_train = pd.read_parquet('c:/Users/tanni/Desktop/rec_sys/Project/prepared_dataset_Nikita/reindexed_data/idea_data/df_clicks_train_indexed')

In [3]:
df_clicks_train.head()

,session,aid,ts,session_encoded,aid_encoded,res_encoded,res_encoded_tfidf,pred,pred_tfidf
0,0,1582117,1660244973157,0,660071,"[756623, 447279, 767847, 309654, 278368, 51127...","[756623, 767847, 278368, 511274, 295778, 30965...","[1813509.0, 1072782.0, 1840418.0, 742592.0, 66...","[1813509.0, 1840418.0, 667924.0, 1226444.0, 70..."
1,1,959544,1659991199405,1,400102,"[723568, 44170, 299744, 89661, 365911, 742127,...","[723568, 89661, 44170, 299744, 356498, 20843, ...","[1734061.0, 105393.0, 718983.0, 215311.0, 8774...","[1734061.0, 215311.0, 105393.0, 718983.0, 8546..."
2,3,54857,1660075780356,2,22877,"[257579, 410617, 22877, 560279, 592905, 3539, ...","[410617, 22877, 257579, 408573, 754463, 240916...","[618373.0, 984459.0, 54857.0, 1343406.0, 14209...","[984459.0, 54857.0, 618373.0, 979687.0, 180835..."
3,6,1329892,1659996712947,3,554595,"[102928, 554595, 599982, 404320, 362899, 77393...","[102928, 554595, 404320, 599982, 44850, 773930...","[247477.0, 1329892.0, 1438169.0, 969717.0, 870...","[247477.0, 1329892.0, 969717.0, 1438169.0, 107..."
4,7,129869,1660222041409,4,54267,"[54267, 39989, 231436, 544107, 353385, 624719,...","[54267, 231436, 39989, 544107, 250768, 292773,...","[129869.0, 95488.0, 555948.0, 1304914.0, 84713...","[129869.0, 555948.0, 95488.0, 1304914.0, 60219..."


In [36]:
df_clicks_train['pred_tfidf'][0]

array([1813509., 1840418.,  667924., 1226444.,  709417.,  742592.,
       1225559., 1072782.,  709550.,  369224., 1407538.,  368671.,
       1816325.,  175982., 1785321.,  197189.,  452936., 1562077.,
       1236087.,  271193.,  279911., 1480079.,  507369., 1369378.,
        481656.,  563653., 1629651.,  528847.,  810311.,  368266.,
        426818., 1840615., 1081927.,  721252., 1554244., 1438491.,
        572667., 1464241., 1852036., 1521766., 1561139.,   27116.,
       1081239., 1791901.,  240346.,  840366.,  268919., 1201592.,
        416502.,  149415.,  729170., 1624381., 1816774.,  977500.,
       1107609.,  141664., 1539771., 1493088., 1251824., 1142924.,
       1774881.,  409656., 1716786., 1335218., 1537102., 1788708.,
       1799950., 1615079.,  810640.,  199983., 1649640., 1048044.,
       1207659.,  819136., 1149615.,  298320.,  137301.,  636540.,
        781953.,  226589.,  357021., 1052813., 1207397., 1531939.,
       1660470.,  431255., 1725503.,  173702., 1214408., 10374

In [12]:
df_clicks_train.to_parquet("c:/Users/tanni/Desktop/rec_sys/Project/prepared_dataset_Nikita/reindexed_data/idea_data/df_clicks_train_for_colab")

In [15]:
df_clicks_train_unique = df_clicks_train.copy()

In [16]:
df_clicks_train_unique['pred'] = df_clicks_train_unique['res_encoded'].apply(clicks_aid_encoder.inverse_transform)

In [ ]:
df_clicks_train_unique['pred_tfidf'] = df_clicks_train_unique['res_encoded_tfidf'].apply(clicks_aid_encoder.inverse_transform)

In [ ]:
df_clicks_train_unique.to_parquet('c:/Users/tanni/Desktop/rec_sys/Project/prepared_dataset_Nikita/reindexed_data/idea_data/df_clicks_train_indexed')

In [2]:
df_clicks_train = pd.read_parquet('c:/Users/tanni/Desktop/rec_sys/Project/prepared_dataset_Nikita/reindexed_data/idea_data/df_clicks_train_indexed')

In [4]:
def set_ranks(df: pd.DataFrame):
    df_c = df.copy()
    df_c["rank"] = df_c.groupby("session")["ts"].rank(method="first", ascending=True)
    return df_c[["session", "aid", "rank"]]

In [5]:
df_clicks_r = set_ranks(df_clicks_train)
#df_carts_r = set_ranks(df_carts_t)
#df_orders_r = set_ranks(df_orders_t)

In [6]:
def rank_stat(df_r: pd.DataFrame, ks: list):
    df_mr = df_r.groupby("session").agg("max")["rank"]
    for k in ks:
        cnt_me = len(df_mr[df_mr >= k])
        print(f"Users with more than {k} test items: {cnt_me} ({round(cnt_me / len(df_mr) * 100, 2)}%)")

In [7]:
rank_stat(df_clicks_r, [3, 5, 10, 20])

Users with more than 3 test items: 0 (0.0%)
Users with more than 5 test items: 0 (0.0%)
Users with more than 10 test items: 0 (0.0%)
Users with more than 20 test items: 0 (0.0%)


In [8]:
#rank_stat(df_carts_r, [3, 5, 10, 20])

In [9]:
#rank_stat(df_orders_r, [3, 5, 10, 20])

# Metrics

In [10]:
K_RECS = 200
K_TEST = 5

In [11]:
df_clicks_r = pd.read_parquet('c:/Users/tanni/Desktop/rec_sys/Project/prepared_dataset_Nikita/clicks/test')

In [12]:
#df_carts_r = pd.read_parquet("data/carts/")

In [13]:
#df_orders_r = pd.read_parquet("data/orders/")

In [14]:
def extract_n_elements(row, n):
    return np.pad(row[:n], (0, max(n - len(row), 0)), mode='constant', constant_values=-1)


In [19]:
def prepare_data(df_test, df_recs, k_test, k_recs):
    valid_session = np.intersect1d(
        df_test["session"].unique(),
        df_recs["session"].unique()
    )

    df_test_l = set_ranks(df_test)
    df_test_l = df_test_l[df_test_l["session"].isin(valid_session)]
    df_test_l = df_test_l[df_test_l["rank"] <= k_test]
    df_test_l = df_test_l.groupby("session")["aid"].apply(list).reset_index().sort_values("session")

    df_test_l["aid"] = (
        df_test_l["aid"]
        .apply(lambda x: np.pad(
            x[:k_test],
            (0, max(k_test - len(x), 0)),
            mode="constant",
            constant_values=-1
        ))
    )
    test = np.stack(df_test_l["aid"].values, axis=0)

    df_recs_l = df_recs[df_recs["session"].isin(valid_session)]
    df_recs_l["modified_pred"] = (
        df_recs_l["pred"]
        .apply(lambda x: extract_n_elements(x, k_recs))
    )
    
    recs = df_recs_l["modified_pred"].tolist()

    return test, np.array(recs)


In [20]:
test_clicks, recs_clicks = prepare_data(df_clicks_r, df_clicks_train, K_TEST, K_RECS)

C:\Users\tanni\AppData\Local\Temp\ipykernel_13840\1135287424.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_recs_l["modified_pred"] = (


In [21]:
recs_clicks[recs_clicks == -1] = 0

In [23]:
recs_clicks.shape

(213272, 200)

In [24]:
import numpy as np


class Metrics:
    def __init__(
            self,
            k: int,
            recommendations: np.ndarray,
            holdout: np.ndarray
            ):
        self.k = k
        self.recommendations = recommendations[:, :k]
        self.holdout = holdout

    def report(self):
        print(f"HR@{self.k} = {round(self.hit_rate(), 4)}")
        print(f"MRR@{self.k} = {round(self.mean_reciprocal_rank(), 4)}")
        print(f"Recall@{self.k} = {round(self.recall(), 4)}")

    def hit_rate(self):
        return (
            self._get_hit_mask()
            .any(axis=1)
            .mean()
        )

    def mean_reciprocal_rank(self):
        hits_mask = self._get_hit_mask()

        idx = np.argwhere(hits_mask.argmax(axis=1)).squeeze(axis=1)

        return np.sum(
            1 / (hits_mask[idx].argmax(axis=1) + 1)
        ) / hits_mask.shape[0]

    def recall(self):
        hits_mask = self._get_hit_mask()
        return (
            hits_mask.sum(axis=1) / self.recommendations.shape[1]
        ).sum() / self.recommendations.shape[0]

    def _get_hit_mask(self) -> np.ndarray:
        return (
            self.holdout[..., None] ==
            np.expand_dims(self.recommendations, axis=1)
        ).any(axis=1)

In [25]:
#from rec_base import Metrics
metrics_carts = Metrics(K_RECS, np.array(recs_clicks), test_clicks)
print(metrics_carts.report())

HR@200 = 0.0411
MRR@200 = 0.0049
Recall@200 = 0.0003
None


# Normalize

In [37]:
def prepare_data(df_test, df_recs, k_test, k_recs):
    valid_session = np.intersect1d(
        df_test["session"].unique(),
        df_recs["session"].unique()
    )

    df_test_l = set_ranks(df_test)
    df_test_l = df_test_l[df_test_l["session"].isin(valid_session)]
    df_test_l = df_test_l[df_test_l["rank"] <= k_test]
    df_test_l = df_test_l.groupby("session")["aid"].apply(list).reset_index().sort_values("session")

    df_test_l["aid"] = (
        df_test_l["aid"]
        .apply(lambda x: np.pad(
            x[:k_test],
            (0, max(k_test - len(x), 0)),
            mode="constant",
            constant_values=-1
        ))
    )
    test = np.stack(df_test_l["aid"].values, axis=0)

    df_recs_l = df_recs[df_recs["session"].isin(valid_session)]
    df_recs_l["modified_pred"] = (
        df_recs_l["pred_tfidf"]
        .apply(lambda x: extract_n_elements(x, k_recs))
    )
    
    recs = df_recs_l["modified_pred"].tolist()

    return test, np.array(recs)


In [38]:
test_clicks, recs_clicks = prepare_data(df_clicks_r, df_clicks_train, K_TEST, K_RECS)

C:\Users\tanni\AppData\Local\Temp\ipykernel_13840\2228420266.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_recs_l["modified_pred"] = (


In [39]:
recs_clicks[recs_clicks == -1] = 0

In [40]:
metrics_carts = Metrics(K_RECS, np.array(recs_clicks), test_clicks)
print(metrics_carts.report())

HR@200 = 0.043
MRR@200 = 0.0056
Recall@200 = 0.0003
None


In [34]:
recs_clicks[1].shape

(200,)